In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)
from sklearn import preprocessing
import pylab as pl
import scipy.optimize as opt
%matplotlib inline

In [2]:
df = pd.read_csv('loan.csv')
df['default'].value_counts()
print('Customers that paid their loans = ',df['default'].value_counts()[1])
print('Customers that NOT paid their loans = ',df['default'].value_counts()[0])

Customers that paid their loans =  142
Customers that NOT paid their loans =  58


In [3]:
X = df[['age', 'ed', 'employ', 'address', 'income', 'debt']] .values  #.astype(float)
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
y = df['default'].values

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (160, 6) (160,)
Test set: (40, 6) (40,)


In [5]:
acscore = np.zeros(4)
jaccard = np.zeros(4)
f1score = np.zeros(4)
logloss = np.zeros(4)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
for n in range(1,Ks):
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1)
k = mean_acc.argmax()+1

The best accuracy was with 0.75 with k= 4


In [7]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat_0 = neigh.predict(X_test)
yhat_prob_0 = neigh.predict_proba(X_test)

In [8]:
from sklearn.tree import DecisionTreeClassifier
loanTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
loanTree.fit(X_train,y_train)
yhat_1 = loanTree.predict(X_test)
yhat_prob_1 = loanTree.predict_proba(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
yhat_2 = LR.predict(X_test)
yhat_prob_2 = LR.predict_proba(X_test)

In [10]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',probability=True)
clf.fit(X_train, y_train) 
yhat_3 = clf.predict(X_test)
yhat_prob_3 = clf.predict_proba(X_test)

In [11]:
from sklearn import metrics
acscore[0] = metrics.accuracy_score(y_test, yhat_0)
acscore[1] = metrics.accuracy_score(y_test, yhat_1)
acscore[2] = metrics.accuracy_score(y_test, yhat_2)
acscore[3] = metrics.accuracy_score(y_test, yhat_3)

In [12]:
from sklearn.metrics import jaccard_score
jaccard[0] = jaccard_score(y_test, yhat_0, average='micro')
jaccard[1] = jaccard_score(y_test, yhat_1, average='micro')
jaccard[2] = jaccard_score(y_test, yhat_2, average='micro')
jaccard[3] = jaccard_score(y_test, yhat_3, average='micro')

In [13]:
from sklearn.metrics import log_loss
logloss[0] = log_loss(y_test, yhat_prob_0)
logloss[1] = log_loss(y_test, yhat_prob_1)
logloss[2] = log_loss(y_test, yhat_prob_2)
logloss[3] = log_loss(y_test, yhat_prob_3)

In [14]:
from sklearn.metrics import f1_score
f1score[0] = f1_score(y_test, yhat_0, average='weighted') 
f1score[1] = f1_score(y_test, yhat_1, average='weighted') 
f1score[2] = f1_score(y_test, yhat_2, average='weighted') 
f1score[3] = f1_score(y_test, yhat_3, average='weighted') 

In [15]:
print('Test set Accuracy = ', acscore)
print('Jaccard score     = ', jaccard)
print('f1-score          = ', f1score)
print('Log loss          = ', logloss)

Test set Accuracy =  [0.75 0.6  0.72 0.62]
Jaccard score     =  [0.6  0.43 0.57 0.45]
f1-score          =  [0.75 0.47 0.7  0.48]
Log loss          =  [2.13 2.13 0.61 0.65]


In [25]:
print( "The best accuracy was with", acscore.max().round(2), "with method ", acscore.argmax()+1)
print( "The best Jaccard score was with", jaccard.max().round(2), "with method ", jaccard.argmax()+1) 
print( "The best f1-score was with", f1score.max().round(2), "with method ", f1score.argmax()+1) 
print( "The best Log loss was with", logloss.min().round(2), "with method ", logloss.argmin()+1) 

The best accuracy was with 0.75 with method  1
The best Jaccard score was with 0.6 with method  1
The best f1-score was with 0.75 with method  1
The best Log loss was with 0.61 with method  3


In [26]:
print("The best classifier is Method 1 that is KNN with the best accuracy, Jaccard and f1-score")

The best classifier is Method 1 that is KNN with the best accuracy, Jaccard and f1-score
